Removing Mitochondria and Chloroplasts from sequences

In [13]:
#get programs to do work (assume we already downloaded the files)
import urllib.request
import shutil
import os

In [14]:
#setup file paths
data_folder=os.path.abspath("../input/data_folder")

In [8]:
#silva and Metaxa2 have already been downloaded from the approptriate websites
#Get fasta files out of Metaxa2 Blast Database
#need to loadt BLAST+
#starting_folder = os.getcwd()
#print(f"Started in working directory: {starting_folder}")
#metaxa2_db_folder = os.path.join(data_folder,"Metaxa2_2.2.3","metaxa2_db","SSU")
#os.chdir(metaxa2_db_folder)
#print("Change folder to ", os.getcwd())

Started in working directory: /home/qiime2/Montipora_live_vs_dead
Change folder to  /home/qiime2/Montipora_live_vs_dead/input/data_folder/Metaxa2_2.2.3/metaxa2_db/SSU


In [16]:
local_silva_filename = "Silva_132_release.zip"
local_silva_filepath = os.path.join(data_folder,local_silva_filename)
from zipfile import is_zipfile, ZipFile

if not is_zipfile("../input/data_folder/Silva_132_release.zip"):
    raise ValueError("The SILVA database zip file {local_silva_filepath} doesn't look like a zip file. Was it downloaded correctly?")

silva_zipfile = ZipFile(local_silva_filepath)

#Obnoxiously this file contains a _MACOSX subfolder. We don't want to unzip that...
files_to_extract = [m for m in silva_zipfile.namelist() if "_MACOSX" not in m]
print("Extracting SILVA database...")
silva_zipfile.extractall(path = data_folder,members = files_to_extract)
silva_zipfile.close()
print(f"Extracted the SILVA 132 database into: {data_folder}")

Extracting SILVA database...
Extracted the SILVA 132 database into: /home/qiime2/Montipora_live_vs_dead/input/data_folder


In [10]:
#generate the fasta from metaxa2 blast database
#print("Generating fasta file from Metaxa2 BLAST database")
#Need to get out of python to do this part
#!blastdbcmd -entry all -db blast -out metaxa2.fasta

#print("Resulting Fasta File(s):")
#!ls ./*.fasta
#!mv metaxa2.fasta $metaxa2_fasta_filepath
#print(f"Moved metaxa2.fasta to {metaxa2_fasta_filepath}")
#os.chdir(starting_folder)
#print("Changed working directory back to: ", os.getcwd())

Generating fasta file from Metaxa2 BLAST database
Resulting Fasta File(s):
./metaxa2.fasta
mv: missing destination file operand after 'metaxa2.fasta'
Try 'mv --help' for more information.


NameError: name 'metaxa2_fasta_filepath' is not defined

Now we will set up the combined Silva Metaxa2 database and run it on the sequences

In [1]:
from Bio import SeqIO
from qiime2 import Artifact
import os

In [2]:
#set up working directory
working_dir = os.path.abspath('../input/data_folder/')

#existing files
silva_fasta_path = working_dir + '/SILVA_132_QIIME_release/rep_set/rep_set_16S_only/99/silva_132_99_16S.fna'
silva_taxonomy_path = working_dir + '/SILVA_132_QIIME_release/taxonomy/16S_only/99/taxonomy_7_levels.txt'
m2_fasta_path = working_dir + '/metaxa2.fasta'

#new intermediate raw files
silva_plus_m2_fasta_path = working_dir + '/m2+silva_otu.fasta'
silva_plus_m2_taxonomy_path = working_dir + '/m2+silva_taxonomy.txt'

#new qiime qza raw files
silva_otu_qza = working_dir + '/silva_otu.qza'
silva_taxonomy = working_dir + '/silva_taxonomy.qza'
silva_m2_otu_qza = working_dir + '/silva_metaxa2_otu.qza'
silva_m2_taxonomy_qza = working_dir + '/silva_metaxa2_taxonomy.qza'

Verify input files

In [3]:
print("Verifying that all needed starting data files exist.")
for existing_file in [silva_fasta_path, silva_taxonomy_path, m2_fasta_path]:
    if not os.path.exists(existing_file):
        raise IOError (f"Required file {existing_file} not found. Please ensure it is in the directory.")
print("Done")

Verifying that all needed starting data files exist.
Done


Supplement SILVA database with metaxa2

In [4]:
#extract mitochondrial sequences from Metaxa2 and create a seperate fasta and taxonomy file in the style of SILVA
otu_file = open(silva_plus_m2_fasta_path, "a")
taxonomy_file = open(silva_plus_m2_taxonomy_path, "a")

for i, entry in enumerate(SeqIO.parse(m2_fasta_path, "fasta")):
    if 'mitochondria' in entry.description or 'Mitochondria' in entry.description:
        otu_file.write(">metaxa2_" + str(i) + "\n")
        otu_file.write(str(entry.seq + "\n"))
        taxonomy_file.write("metaxa2_" + str(i) + "\tD_O_Bacteria;D_1_Proteobacteria;D_2_Alphaproteobacteria;D_3_Rickettsiales;D_4_Mitochondria;D_5_uncultured bacterium;D_6_uncultured bacterium\n")
    else:
        continue

In [5]:
otu_file = open(silva_plus_m2_fasta_path, "a")
#copy silva otus into the combined fasta file 
for entry in SeqIO.parse(silva_fasta_path, "fasta"):
    otu_file.write(">" + str(entry.description) + "\n")
    otu_file.write(str(entry.seq) + "\n")
    #print(">" + str(entry.description) +"\n")
    #print(str(entry.seq)+"\n")
   

In [31]:
#copy Silva OTUs into combined fasta file
#for entry in SeqIO.parse(silva_fasta_path, "fasta"):
 #   otu_file.write(">" + str(entry.description + "\n"))
  #  otu_file.write(str(entry.seq + "\n"))

In [6]:
#copy silva Taxonomy to new file
silva_taxonomy_file = open(silva_taxonomy_path)
for line in silva_taxonomy_file:
    taxonomy_file.write(line)

In [7]:
#verify we created the files

print("Verify that all silva_plus_metaxa2 data files were created")
for created_file in [silva_plus_m2_fasta_path, silva_plus_m2_taxonomy_path]:
    if not os.path.exists(created_file):
        raise IOError (f"Required file {created_file} not found. Please ensure it is in the directory.")
    print("Sucessfully created file:", created_file)
    
print("Done")

Verify that all silva_plus_metaxa2 data files were created
Sucessfully created file: /home/qiime2/Montipora_live_vs_dead/input/data_folder/m2+silva_otu.fasta
Sucessfully created file: /home/qiime2/Montipora_live_vs_dead/input/data_folder/m2+silva_taxonomy.txt
Done


Import the Silva + Metaxa2 files into Qiime2 and export as qza files

In [8]:
#create Silva taxonomy and ASV artifacts
silva_otu = Artifact.import_data('FeatureData[Sequence]', silva_fasta_path)
silva_taxonomy = Artifact.import_data('FeatureData[Taxonomy]', silva_taxonomy_path, 'HeaderlessTSVTaxonomyFormat')

In [9]:
#create silva+m2 taxonomy and ASV artifacts
silva_m2_otu = Artifact.import_data('FeatureData[Sequence]', silva_plus_m2_fasta_path)
silva_m2_taxonomy = Artifact.import_data('FeatureData[Taxonomy]', silva_plus_m2_taxonomy_path, 'HeaderlessTSVTaxonomyFormat')

In [10]:
#export the artifacts to the output folder
silva_otu.save('silva_otu.qza')
silva_taxonomy.save('silva_taxonomy.qza')
silva_m2_otu.save('silva_m2_otu.qza')
silva_m2_taxonomy.save('silva_m2_taxonomy.qza')

'silva_m2_taxonomy.qza'